In [5]:
import sys,time
import import_ipynb
from uszipcode import SearchEngine

In [9]:
%run ../api/classy990downloads.ipynb


year = 2017
uri='mongodb://mongo/irs990'

db = mongo990( uri=uri, year=year, dump=False)
        
db.verify()



Connect to: [mongodb://mongo/irs990]
List of databases in MongoDB: ['admin', 'config', 'irs990', 'local']
Open db: [irs990]
Collection: [2017]
1170 documents present
Close connection to [irs990] [2017]
Success on close for [2017]
1170 documents present


In [10]:
search = SearchEngine(simple_zipcode=True) # Load search Engine
for x in db.collection.find( {},{ "_id": 1, "/IRS990/USAddress/ZIPCode": 1, 'OrganizationName':1 } ):
    name = x['OrganizationName']
    try:
        zipcode = x['/IRS990/USAddress/ZIPCode']
    except:
        zipcode = False
    
    
    padding = '                                              '
    sys.stdout.write( '%s%s%s\r' % (padding,padding,padding) )
    sys.stdout.flush() 
    
    
    sys.stdout.write( '%s => %s ' % ( name, zipcode ) )

    if not zipcode or 'N/A' in zipcode or 'n/a' in zipcode:
        sys.stdout.write( '*** MISSING zipcode *** ')
        
    else:
        county = search.by_zipcode(zipcode).county

        if county:
            try:
                sys.stdout.write(x['_id'])
                sys.stdout.write(county)
                db.collection.update_one( {"_id" : x['_id']}, {"$set":{'County':county}}, upsert=True )
            except Exception as e:
                sys.stdout.write( 'ERROR: Problem during MongoDB insert. %s' % e )

        else:
            sys.stdout.write( ' *** NO DATA ***' )

    sys.stdout.write( '\r' )
    sys.stdout.flush() 